In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['X_train.csv', 'sample_submission.csv', 'X_test.csv', 'y_train.csv']


In [2]:
df_X_train = pd.read_csv("../input/X_train.csv")
df_X_test = pd.read_csv("../input/X_test.csv")
df_y_train = pd.read_csv("../input/y_train.csv")

def check_for_nas(df):
    return sum(df.isnull().sum())

print("Number of na values in X_train: {}".format(check_for_nas(df_X_train)))
print("Number of na values in X_test: {}".format(check_for_nas(df_X_test)))
print("Number of na values in y_train: {}".format(check_for_nas(df_y_train)))

Number of na values in X_train: 0
Number of na values in X_test: 0
Number of na values in y_train: 0


Check surface value counts

In [3]:
df_y_train['surface'].value_counts()

concrete                  779
soft_pvc                  732
wood                      607
tiled                     514
fine_concrete             363
hard_tiles_large_space    308
soft_tiles                297
carpet                    189
hard_tiles                 21
Name: surface, dtype: int64

Imbalance is apparent - need to think about this later

In [4]:
def convert_q_e(list_):
    '''Convert quaternion to euler angles.
    list_ has 4 elements in the order mentioned below
    or_x - X orientation
    or_y - Y orientation
    or_z - Z orientation
    or_w - W orientation
    Returns a tuple of (roll,pitch,yaw)'''
    or_x = list_[0]
    or_y = list_[1] 
    or_z = list_[2] 
    or_w = list_[3] 
    
    sinr_cosp = 2 * ((or_w * or_x) + (or_y * or_z))
    cosr_cosp = 1 - 2 * ((or_x * or_x) + (or_y * or_y))
    roll = np.arctan2(sinr_cosp, cosr_cosp)
    
    pitch = None
    sinp = 2 * ((or_w * or_y) - (or_z * or_x))
    if abs(sinp) >= 1:
        pitch = np.copysign(np.pi/2,sinp)
    else:
        pitch = np.arcsin(sinp)
    
    siny_cosp = 2 * ((or_w * or_z) + (or_x * or_y))
    cosy_cosp = 1 - 2 * ((or_y * or_y) + (or_z * or_z))
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    
    return(roll, pitch, yaw)



Replace orientaion_x/y/z/w in the dataframe with euler angles in both training and test sets

In [5]:
# Using apply to convert orientation to euler angles
orientation_col_names = ['orientation_X','orientation_Y','orientation_Z','orientation_W']
df_X_train_euler = df_X_train.loc[:,orientation_col_names].apply(convert_q_e,axis=1,result_type='expand')
df_X_train_euler.rename(columns={0:'roll',1:'pitch',2:'yaw'},inplace = True)

df_X_test_euler = df_X_test.loc[:,orientation_col_names].apply(convert_q_e,axis=1,result_type='expand')
df_X_test_euler.rename(columns={0:'roll',1:'pitch',2:'yaw'}, inplace=True)
    
# df_X_train_processed = df_X_train.drop(orientation_col_names,axis=1)
df_X_train_processed = pd.concat([df_X_train,df_X_train_euler], axis = 1)
# Need to remove row_id and measurement_number as they are redundant
redundant_cols = ['row_id','measurement_number']
df_X_train_processed = df_X_train_processed.drop(redundant_cols,axis=1)

# df_X_test_processed = df_X_test.drop(orientation_col_names,axis=1)
df_X_test_processed = pd.concat([df_X_test,df_X_test_euler],axis = 1)
df_X_test_processed = df_X_test_processed.drop(redundant_cols, axis = 1)
   

In [6]:
df_X_test_processed.head()

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,roll,pitch,yaw
0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439,2.844733,-0.014254,3.091597
1,0,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273,2.844432,-0.014411,3.091804
2,0,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990,2.844239,-0.014658,3.091975
3,0,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270,2.844418,-0.014712,3.092085
4,0,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,-10.4100,2.844918,-0.014434,3.092079


## Feature engineering (Need to add many more features)

Series Ids need to be grouped by and channels that have sensor information from the physical work need to be aggregated. Following ideas for aggregating sensor values need to be tested
1. angular velocity integration - this gives the angular distance traveled for the group - assume 1 millisecond delta for this calculation
2. Mean of absolute of linear acceleration - as a proxy for acceleration and braking for the surface
3. sum of absolute diffs of roll/pitch/yaw - Gives the total angle covered in each dimension - proxy to swiftness
4. Angular_velocity diff means - as a proxy to velocity angular velocity changes, mostly for changes in direction
5. Means of all columns
6. Standard Devs of all columns
7. Max of all columns
8. Min
9. Range
10. Median
11. Diff of Linear acceleration is a proxy for jerk - traaction and inertia are functions of jerk


** In this run - not using cols_affected and letting transformations work on every channel **

In [7]:
# Function to address 1
integration = lambda x: 0.001 * (x.diff().sum())

# Function to address 2
mean_abs = lambda x:x.abs().mean()

# Function to address 3
sum_abs = lambda x:x.abs().sum()

# Function to address 4
mean_diff = lambda x:x.mean()

# Function to address 11
def modified_mean_diff(x):
    res = [0]
    res.extend(list(np.diff(x)))
    return np.mean(res)


In [8]:
# Function to create aggregates
def create_aggregates(df,function,appendage,cols_affected = None):
    '''Returns an aggregate dataset based on group_by dataset. Appendage modifies colnames by adding this string to the end of the colname
    df - Pandas Dataframe Object on which group by is performed
    cols_affected - list of columns on which aggregation needs to be performed
    function - function to aggregate groupby df on
    appendage - string to modify colnames'''
    if cols_affected:
        cols_affected.append('series_id')
        df_sub = df.loc[:,cols_affected].copy()
        group_by_df = df_sub.groupby(by=['series_id'])
    else:
        group_by_df = df.groupby(by=['series_id'])
    
    df_ = group_by_df.agg(function).rename(mapper = lambda x:x + "_" + appendage, axis = 'columns').copy()
    return df_
    

In [9]:
# Create feature datasets based on bullet points above. Merge them later as a final step
#Step - 1
#Train
df_X_train_step_1 = create_aggregates(df_X_train_processed,function=integration,appendage='integral')
#Test
df_X_test_step_1 = create_aggregates(df_X_test_processed,function=integration,appendage='integral')

#Step - 2
#Train
df_X_train_step_2 = create_aggregates(df_X_train_processed,function=mean_abs,appendage='mean_abs')
#Test
df_X_test_step_2 = create_aggregates(df_X_test_processed,function=mean_abs,appendage='mean_abs')

#Step - 3
#Train
df_X_train_step_3 = create_aggregates(df_X_train_processed,function=sum_abs,appendage='sum_abs')
#Test
df_X_test_step_3 = create_aggregates(df_X_test_processed,function=sum_abs,appendage='sum_abs')

#Step - 4
#Train
df_X_train_step_4 = create_aggregates(df_X_train_processed,function=mean_diff,appendage='mean_diff')
#Test
df_X_test_step_4 = create_aggregates(df_X_test_processed,function=mean_diff,appendage='mean_diff')

#Step - 5
#Train
df_X_train_step_5 = create_aggregates(df_X_train_processed,function=np.mean,appendage='mean')
#Test
df_X_test_step_5 = create_aggregates(df_X_test_processed,function=np.mean,appendage='mean')

#Step - 6
#Train
df_X_train_step_6 = create_aggregates(df_X_train_processed,function=np.std,appendage='std')
#Test
df_X_test_step_6 = create_aggregates(df_X_test_processed,function=np.std,appendage='std')

#Step - 7 
#Train
df_X_train_step_7 = create_aggregates(df_X_train_processed,function=np.max,appendage='max')
#Test
df_X_test_step_7 = create_aggregates(df_X_test_processed,function=np.max,appendage='max')

#Step - 8
#Trainm
df_X_train_step_8 = create_aggregates(df_X_train_processed,function=np.min,appendage='min')
#Test
df_X_test_step_8 = create_aggregates(df_X_test_processed,function=np.min,appendage='min')

# Step-9
#Train
df_X_train_step_9 = create_aggregates(df_X_train_processed,function=np.ptp,appendage='ptp')
#Test
df_X_test_step_9 = create_aggregates(df_X_test_processed,function=np.ptp,appendage='ptp')

# Step - 10
#Train
df_X_train_step_10 = create_aggregates(df_X_train_processed,function=np.median,appendage='median')
#Test
df_X_test_step_10 = create_aggregates(df_X_test_processed,function=np.median,appendage='median')

# Step - 11
#Train
df_X_train_step_11 = create_aggregates(df_X_train_processed,function=modified_mean_diff,appendage='mod_diff')
#Test
df_X_test_step_11 = create_aggregates(df_X_test_processed,function=modified_mean_diff,appendage='mod_diff')

In [10]:
X_train_master = pd.concat([df_X_train_step_1,df_X_train_step_2,df_X_train_step_3,df_X_train_step_4,df_X_train_step_5,df_X_train_step_6,df_X_train_step_7,df_X_train_step_8,df_X_train_step_9,df_X_train_step_10,df_X_train_step_11],axis=1,join='inner')
X_test_master = pd.concat([df_X_test_step_1,df_X_test_step_2,df_X_test_step_3,df_X_test_step_4,df_X_test_step_5,df_X_test_step_6,df_X_test_step_7,df_X_test_step_8,df_X_test_step_9,df_X_test_step_10,df_X_test_step_11],axis=1,join='inner')

In [11]:
X_train_master.head()

,orientation_X_integral,orientation_Y_integral,orientation_Z_integral,orientation_W_integral,angular_velocity_X_integral,angular_velocity_Y_integral,angular_velocity_Z_integral,linear_acceleration_X_integral,linear_acceleration_Y_integral,linear_acceleration_Z_integral,roll_integral,pitch_integral,yaw_integral,orientation_X_mean_abs,orientation_Y_mean_abs,orientation_Z_mean_abs,orientation_W_mean_abs,angular_velocity_X_mean_abs,angular_velocity_Y_mean_abs,angular_velocity_Z_mean_abs,linear_acceleration_X_mean_abs,linear_acceleration_Y_mean_abs,linear_acceleration_Z_mean_abs,roll_mean_abs,pitch_mean_abs,yaw_mean_abs,orientation_X_sum_abs,orientation_Y_sum_abs,orientation_Z_sum_abs,orientation_W_sum_abs,angular_velocity_X_sum_abs,angular_velocity_Y_sum_abs,angular_velocity_Z_sum_abs,linear_acceleration_X_sum_abs,linear_acceleration_Y_sum_abs,linear_acceleration_Z_sum_abs,roll_sum_abs,pitch_sum_abs,yaw_sum_abs,orientation_X_mean_diff,...,yaw_min,orientation_X_ptp,orientation_Y_ptp,orientation_Z_ptp,orientation_W_ptp,angular_velocity_X_ptp,angular_velocity_Y_ptp,angular_velocity_Z_ptp,linear_acceleration_X_ptp,linear_acceleration_Y_ptp,linear_acceleration_Z_ptp,roll_ptp,pitch_ptp,yaw_ptp,orientation_X_median,orientation_Y_median,orientation_Z_median,orientation_W_median,angular_velocity_X_median,angular_velocity_Y_median,angular_velocity_Z_median,linear_acceleration_X_median,linear_acceleration_Y_median,linear_acceleration_Z_median,roll_median,pitch_median,yaw_median,orientation_X_mod_diff,orientation_Y_mod_diff,orientation_Z_mod_diff,orientation_W_mod_diff,angular_velocity_X_mod_diff,angular_velocity_Y_mod_diff,angular_velocity_Z_mod_diff,linear_acceleration_X_mod_diff,linear_acceleration_Y_mod_diff,linear_acceleration_Z_mod_diff,roll_mod_diff,pitch_mod_diff,yaw_mod_diff
series_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-1.000000e-06,0.000001,2.700000e-07,-8.700000e-07,-0.000034,0.000003,0.000018,0.000214,-0.001300,-0.000944,-9.493624e-07,1.028808e-06,-0.000003,0.758666,0.634008,0.105474,0.106470,0.041248,0.026411,0.014996,0.764930,2.984195,9.320391,2.841734,0.025037,1.396035,97.10922,81.15298,13.500690,13.628110,5.279737,3.380616,1.919517,97.911042,381.976947,1193.0100,363.741973,3.204732,178.692530,-0.758666,...,1.393233,0.00131,0.00150,0.001530,0.001460,0.268060,0.152102,0.081901,4.71820,5.310983,6.2439,0.003720,0.002233,0.004169,-0.758530,-0.634270,-0.105500,-0.106555,-0.005082,-0.004037,0.006842,0.231665,3.40755,-9.42995,2.841582,-0.025049,1.396673,-0.000008,0.000010,0.000002,-0.000007,-0.000264,0.000022,0.000138,0.001674,-0.010153,-0.007373,-0.000007,8.037563e-06,-0.000027
1,3.200000e-07,0.000002,1.238000e-06,7.400000e-07,-0.000212,0.000029,0.000108,-0.002178,-0.000180,-0.000753,8.629812e-07,2.255212e-06,-0.000004,0.958606,0.241867,0.031650,0.146876,0.072018,0.035467,0.058881,0.796827,2.873462,9.388899,2.840129,0.010369,0.492731,122.70162,30.95897,4.051239,18.800070,9.218290,4.539781,7.536752,101.993821,367.803192,1201.7791,363.536528,1.327174,63.069540,-0.958606,...,-0.494641,0.00059,0.00196,0.001837,0.002220,0.538220,0.246410,0.250760,8.29360,8.834200,14.1831,0.004715,0.002964,0.004477,-0.958595,0.241890,0.031688,-0.146910,0.010344,-0.006330,0.003538,0.003571,2.75010,-9.41380,2.840183,-0.010446,-0.492860,0.000003,0.000012,0.000010,0.000006,-0.001656,0.000224,0.000844,-0.017017,-0.001403,-0.005886,0.000007,1.761885e-05,-0.000028
2,-4.900000e-06,0.000003,9.000000e-07,-9.040000e-07,0.000079,-0.000006,-0.000059,-0.002076,-0.000574,-0.002513,6.939526e-07,7.779561e-07,-0.000012,0.512057,0.846171,0.129371,0.071082,0.048996,0.028078,0.029764,0.613776,2.951067,9.395783,2.845529,0.012195,2.055021,65.54334,108.30988,16.559440,9.098478,6.271455,3.594032,3.809751,78.563381,377.736515,1202.6602,364.227651,1.560956,263.042744,-0.512057,...,2.049684,0.00490,0.00289,0.001780,0.001157,0.294630,0.199756,0.104427,4.44630,7.464500,6.7548,0.003743,0.000939,0.011604,-0.512035,-0.846210,-0.129405,-0.071

In [12]:
# Scale the datasets before proceeding to classification tasks
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
X_train_master_scaled = scaler.fit_transform(X_train_master)
X_test_master_scaled = scaler.fit_transform(X_test_master)


In [13]:
# Y_train is not numeric yet - convert using LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_labels_encoded = le.fit_transform(df_y_train['surface'])

## Try a GDBT/RandomForest
Trying on a GDBT before moving on to Neural Nets (Assuming Neural Nets are superior for this problem)



In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# param_grid = {'n_estimators':[500],'max_depth':[5],'max_features':['log2']}

# param_grid = {'n_estimators':[500]}
gdbt = GradientBoostingClassifier()

# gscv = GridSearchCV(estimator=gdbt,param_grid=param_grid,cv=5)
gscv = GradientBoostingClassifier(n_estimators=500,max_depth=5,max_features='log2')
gscv.fit(X_train_master_scaled,y_labels_encoded)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features='log2', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [15]:
# print("Best score: {}".format(gscv.best_score_))
# print("Best Classifier: {}".format(gscv.best_estimator_))

In [16]:
import csv
def write_to_csv(test_array,classifier,encoder,output_file_name="predictions.csv"):
    '''Writes a sample csv
    test_array - numpy array; this needs to be sorted on series_id
    classifier - result from cross vaidation analysis
    encoder - label encoder used for labels'''
    predictions = classifier.predict(test_array)
    transformed_predictions = list((encoder.inverse_transform(predictions)))
    series_id = list(range(len(transformed_predictions)))
    df_ = pd.DataFrame({'series_id':series_id,'surface':transformed_predictions})
    df_.to_csv(output_file_name,index=False)
    return df_
    
# write_to_csv(X_test_master_scaled,gscv.best_estimator_,le)
write_to_csv(X_test_master_scaled,gscv,le)

,series_id,surface
0,0,tiled
1,1,carpet
2,2,tiled
3,3,wood
4,4,soft_tiles
5,5,concrete
6,6,concrete
7,7,concrete
8,8,wood
9,9,wood


## Softmax regression using tensor-flow
Idea is to build a neural-net in tensor-flow and perform softmax regression. Therefore, one-hot encoding needs to be performed so that each of the surface types is assigned a bit on a 9-bit vector

concrete - [1 0 0 0 0 0 0 0 0] <br/>
soft_pvc = [0 1 0 0 0 0 0 0 0] <br/>
.
.
so on

Start with a 2 - layer 128 -> 9 hidden unit neural network


In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# surface_labels = np.array(df_y_train['surface']).reshape(-1,1)
# ohe = OneHotEncoder(handle_unknown='ignore')
# surface_labels_ohe = ohe.fit_transform(surface_labels).toarray()

In [ ]:
# import tensorflow as tf

# x = tf.placeholder(tf.float32,[None,X_train_master.shape[1]])

# W1 = tf.Variable(tf.random.normal([X_train_master.shape[1],256]))
# b1 = tf.Variable(tf.zeros([1,256]))
# Z1 = tf.matmul(x,W1) + b1
# sigma1 = tf.nn.relu(Z1)

# W2 = tf.Variable(tf.random.normal([256,9]))
# b2 = tf.Variable(tf.zeros([1,9]))
# Z2 = tf.matmul(sigma1, W2) + b2
# sigma2 = tf.nn.sigmoid(Z2)

# y = tf.nn.softmax(sigma2)
# y_ = tf.placeholder(tf.float32,[None,surface_labels_ohe.shape[1]])



# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# train_step = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cross_entropy)


# init = tf.global_variables_initializer()

# max_iter = 10000

# with tf.Session() as sess:
#     sess.run(init)
#     iters = 0
#     while iters < max_iter:
# #         for i in range(X_train_master_scaled.shape[0]):
# #         print(sess.run(W1,feed_dict={x:X_train_master_scaled[i,:].reshape(1,93),y_:surface_labels_ohe[i,:].reshape(1,9)}))
#         sess.run(train_step,feed_dict={x:X_train_master_scaled.reshape(-1,93),y_:surface_labels_ohe.reshape(-1,9)})
# #         print(sess.run(cross_entropy,feed_dict={x:X_train_master_scaled.reshape(-1,93),y_:surface_labels_ohe.reshape(-1,9)}))
#         iters = iters + 1
    
#     prediction = tf.argmax(y,1)
#     predicted_labels = prediction.eval(feed_dict={x: X_train_master_scaled.reshape(-1,93)})
#     correct_prediction = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(y_,1)),tf.float32)) / 3810
#     print(correct_prediction.eval(feed_dict={x:X_train_master_scaled.reshape(-1,93),y_:surface_labels_ohe.reshape(-1,9)}))
    
#     print(prediction.eval(feed_dict={x: X_train_master_scaled.reshape(-1,93)}))
    
# y = tf.nn.softmax(tf.matmul(x,W) + b)

# y_ = tf.placeholder(tf.float32,[None,surface_labels_ohe.shape[1]])

# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# sess = tf.Session()
# tf.global_variables_initializer().run(session=sess)

# train_features = X_train_master[10000:]
# train_labels = surface_labels_ohe[10000:]

# for _ in range(1000):
#   batch_xs, batch_ys = train_features, train_labels
#   sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    
# sess.close()
# # correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))


In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
prediction = tf.argmax(y,1)
predicted_labels = prediction.eval(feed_dict={x: train_features})
# print(prediction.eval(feed_dict={x: train_features}))


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
w = tf.Variable([0],dtype=tf.float32)
x = tf.placeholder(tf.float32,[3,1])
cost = x[0][0] * w ** 2 + x[1][0] * w + x[2][0]
coeff = np.array([[1],[-10],[25]])
W1 = tf.Variable(tf.random.normal([X_train_master.shape[1],128]))

train = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(W1))